In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pickle
import joblib

#preprocess
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# modeling
import catboost as ctb
import itertools #grid search用
from sklearn.model_selection import KFold

#metrics
from sklearn.metrics import mean_squared_error

#progress bar
from tqdm.notebook import tqdm

import sys
sys.path.append('../src')
from util import simple_preprocess
from LGBM import lgb_modelling
#plot setting
import myplot
myplot.setting_myplot()

In [19]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',100)
pd.set_option("display.max_seq_item",150)
pd.set_option("display.width",None)

In [20]:
os.listdir('../features/')

['0.26986_feature.pkl',
 '0.26993_feature.pkl',
 '0.26999_feature.pkl',
 'base.pkl',
 '前面道路の方位区分.pkl',
 '前面道路の方位区分_取引年.pkl',
 '前面道路区分.pkl',
 '前面道路区分_取引年.pkl',
 '形状区分.pkl',
 '形状区分_取引年.pkl',
 '所在地コード.pkl',
 '所在地コード_取引年.pkl',
 '用途区分.pkl',
 '用途区分_取引年.pkl',
 '駅名.pkl',
 '駅名_取引年.pkl']

In [21]:
with open('../features/base.pkl', 'rb') as f:
    full = pickle.load(f)

merge_file = [
    '前面道路の方位区分_取引年.pkl',
     '前面道路区分_取引年.pkl',
     '形状区分_取引年.pkl',
     '所在地コード_取引年.pkl',
     '用途区分_取引年.pkl',
     '駅名_取引年.pkl']

for file in merge_file:
    with open('../features/'+file, 'rb') as f:
        df = pickle.load(f)
        full = pd.concat([full,df],axis=1)

In [22]:
full.columns

Index(['id', '種類', '地域', '市区町村コード', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り',
       '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的',
       '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点',
       '改装', '取引の事情等', 'y', 'adversarial', '間取り_Ｌ', '間取り_Ｄ', '間取り_Ｋ', '間取り_＋Ｓ',
       '間取り_Ｒ', '間取り_メゾネット', '間取り_オープンフロア', '間取り_スタジオ', '間取り_＋Ｋ',
       '間取り_missing', '間取り_num', '面積（㎡）_2000㎡以上', '土地の形状_rank', '建築年_西暦',
       '建物の構造_木造', '建物の構造_鉄骨造', '建物の構造_ＳＲＣ', '建物の構造_ＲＣ', '建物の構造_missing',
       '建物の構造_ブロック造', '建物の構造_軽量鉄骨造', '取引時点_年', '取引時点_四半期', '取引の事情等_関係者間取引',
       '取引の事情等_その他事情有り', '取引の事情等_他の権利・負担付き', '取引の事情等_missing', '取引の事情等_隣地の購入',
       '取引の事情等_瑕疵有りの可能性', '取引の事情等_調停・競売等', '取引の事情等_私道を含む取引', '用途_その他', '用途_倉庫',
       '用途_店舗', '用途_住宅', '用途_駐車場', '用途_作業場', '用途_工場', '用途_missing', '用途_共同住宅',
       '用途_事務所', '経過年月', '前面道路の方位区分_mean_取引年', '前面道路の方位区分_std_取引年',
       '前面道路の方位区分_count_取引年', '前面道路の方位区分_median_取引年', '前面道路区分_mean_取引年',
       '前面道路区分_std_取引年'

In [23]:
use_columns = ['id', '種類', '地域', '市区町村コード', '地区名', '最寄駅：名称', '最寄駅：距離（分）',
       '面積（㎡）', '間口', '延床面積（㎡）', '今後の利用目的',
       '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '改装',
       'y', 'adversarial', '間取り_Ｌ', '間取り_Ｄ', '間取り_Ｋ', '間取り_＋Ｓ',
       '間取り_Ｒ', '間取り_メゾネット', '間取り_オープンフロア', '間取り_スタジオ', '間取り_＋Ｋ',
       '間取り_missing', '間取り_num', '面積（㎡）_2000㎡以上', '土地の形状_rank', '建築年_西暦',
       '建物の構造_ＳＲＣ', '建物の構造_ブロック造', '建物の構造_軽量鉄骨造', '建物の構造_ＲＣ', '建物の構造_木造',
       '建物の構造_鉄骨造', '建物の構造_missing', '取引時点_年', '取引時点_四半期', '取引の事情等_関係者間取引',
       '取引の事情等_瑕疵有りの可能性', '取引の事情等_その他事情有り', '取引の事情等_他の権利・負担付き',
       '取引の事情等_私道を含む取引', '取引の事情等_隣地の購入', '取引の事情等_missing', '取引の事情等_調停・競売等',
       '用途_工場', '用途_事務所', '用途_住宅', '用途_駐車場', '用途_共同住宅', '用途_店舗', '用途_倉庫',
       '用途_作業場', '用途_missing', '用途_その他','経過年月',
       '所在地コード_mean_取引年', '所在地コード_std_取引年', '所在地コード_count_取引年',
       '所在地コード_median_取引年', '駅名_mean_取引年', '駅名_std_取引年', '駅名_count_取引年',
       '駅名_median_取引年']
full = full[use_columns]

In [24]:
train = full[full['adversarial']==0]
test = full[full['adversarial']==1]
del full

In [25]:
pp_full,col_ins = simple_preprocess(train,test,"id","y")

object型のユニーク数： 種類 5 地域 5 地区名 1455 最寄駅：名称 657 今後の利用目的 7 前面道路：方位 10 前面道路：種類 15 都市計画 17 改装 3 
----------
shape: (356344, 70)
objects: 9


,種類,地域,地区名,最寄駅：名称,今後の利用目的,前面道路：方位,前面道路：種類,都市計画,改装
0,0,0,1404,628,3,0,0,1,1
1,0,0,1404,628,3,0,0,1,2
2,0,0,1404,628,1,0,0,1,2


floats: 6


,間口,前面道路：幅員（ｍ）,所在地コード_mean_取引年,所在地コード_std_取引年,駅名_mean_取引年,駅名_std_取引年
0,NaN,NaN,5.757475e+06,7.547589e+06,2062500.0,1.101737e+06
1,NaN,NaN,5.757475e+06,7.547589e+06,2062500.0,1.101737e+06
2,NaN,NaN,5.757475e+06,7.547589e+06,2062500.0,1.101737e+06


ints: 38


,市区町村コード,adversarial,間取り_Ｌ,間取り_Ｄ,間取り_Ｋ,間取り_＋Ｓ,間取り_Ｒ,間取り_メゾネット,間取り_オープンフロア,間取り_スタジオ,間取り_＋Ｋ,間取り_missing,面積（㎡）_2000㎡以上,建物の構造_ＳＲＣ,建物の構造_ブロック造,建物の構造_軽量鉄骨造,建物の構造_ＲＣ,建物の構造_木造,建物の構造_鉄骨造,建物の構造_missing,取引の事情等_関係者間取引,取引の事情等_瑕疵有りの可能性,取引の事情等_その他事情有り,取引の事情等_他の権利・負担付き,取引の事情等_私道を含む取引,取引の事情等_隣地の購入,取引の事情等_missing,取引の事情等_調停・競売等,用途_工場,用途_事務所,用途_住宅,用途_駐車場,用途_共同住宅,用途_店舗,用途_倉庫,用途_作業場,用途_missing,用途_その他
0,13101,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
1,13101,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0
2,13101,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0


float to int 15


,最寄駅：距離（分）,面積（㎡）,延床面積（㎡）,建ぺい率（％）,容積率（％）,間取り_num,土地の形状_rank,建築年_西暦,取引時点_年,取引時点_四半期,経過年月,所在地コード_count_取引年,所在地コード_median_取引年,駅名_count_取引年,駅名_median_取引年
0,1.0,55.0,NaN,80.0,600.0,2.0,9.0,1984.0,2017.0,4.0,33.0,61.0,2720000.0,4.0,1795000.0
1,5.0,20.0,NaN,80.0,500.0,1.0,9.0,2003.0,2017.0,4.0,14.0,61.0,2720000.0,4.0,1795000.0
2,3.0,45.0,NaN,80.0,500.0,1.0,9.0,2012.0,2017.0,3.0,5.0,61.0,2720000.0,4.0,1795000.0


bool型の列数 0
all: 68
int型とfloatだけどint型のユニーク数
市区町村コード 59 adversarial 2 間取り_Ｌ 2 間取り_Ｄ 2 間取り_Ｋ 2 間取り_＋Ｓ 2 間取り_Ｒ 2 間取り_メゾネット 2 間取り_オープンフロア 2 間取り_スタジオ 2 間取り_＋Ｋ 2 間取り_missing 2 面積（㎡）_2000㎡以上 2 建物の構造_ＳＲＣ 2 建物の構造_ブロック造 2 建物の構造_軽量鉄骨造 2 建物の構造_ＲＣ 2 建物の構造_木造 2 建物の構造_鉄骨造 2 建物の構造_missing 2 取引の事情等_関係者間取引 2 取引の事情等_瑕疵有りの可能性 2 取引の事情等_その他事情有り 2 取引の事情等_他の権利・負担付き 2 取引の事情等_私道を含む取引 2 取引の事情等_隣地の購入 2 取引の事情等_missing 2 取引の事情等_調停・競売等 2 用途_工場 2 用途_事務所 2 用途_住宅 2 用途_駐車場 2 用途_共同住宅 2 用途_店舗 2 用途_倉庫 2 用途_作業場 2 用途_missing 2 用途_その他 2 最寄駅：距離（分） 35 面積（㎡） 157 延床面積（㎡） 130 建ぺい率（％） 7 容積率（％） 18 間取り_num 8 土地の形状_rank 10 建築年_西暦 76 取引時点_年 15 取引時点_四半期 4 経過年月 78 所在地コード_count_取引年 112 所在地コード_median_取引年 561 駅名_count_取引年 33 駅名_median_取引年 2179 

In [26]:
categories = ['市区町村コード',
              '間取り_Ｌ','間取り_Ｄ', '間取り_Ｋ', '間取り_＋Ｓ', '間取り_Ｒ', '間取り_メゾネット', '間取り_オープンフロア',
              '間取り_スタジオ', '間取り_＋Ｋ', '間取り_missing',
              '建物の構造_木造', '建物の構造_missing', '建物の構造_鉄骨造', '建物の構造_ＲＣ',
              '建物の構造_ＳＲＣ', '建物の構造_軽量鉄骨造', '建物の構造_ブロック造',
              '取引の事情等_調停・競売等', '取引の事情等_私道を含む取引', '取引の事情等_missing', '取引の事情等_関係者間取引',
              '取引の事情等_その他事情有り', '取引の事情等_他の権利・負担付き',
              '取引の事情等_隣地の購入', '取引の事情等_瑕疵有りの可能性',
              '用途_作業場', '用途_その他', '用途_missing','用途_倉庫', '用途_共同住宅', '用途_店舗', 
              '用途_住宅', '用途_駐車場', '用途_工場', '用途_事務所',
              '面積（㎡）_2000㎡以上'
             ]
for col in categories:
    pp_full[col] = pp_full[col].astype("category")

In [27]:
print(pp_full.select_dtypes('category').columns)
print(pp_full.select_dtypes('int64').columns)
print(pp_full.select_dtypes(float).columns)

train = pp_full[pp_full['adversarial']==0]
test = pp_full[pp_full['adversarial']==1]

Index(['種類', '地域', '市区町村コード', '地区名', '最寄駅：名称', '今後の利用目的', '前面道路：方位', '前面道路：種類',
       '都市計画', '改装', '間取り_Ｌ', '間取り_Ｄ', '間取り_Ｋ', '間取り_＋Ｓ', '間取り_Ｒ', '間取り_メゾネット',
       '間取り_オープンフロア', '間取り_スタジオ', '間取り_＋Ｋ', '間取り_missing', '面積（㎡）_2000㎡以上',
       '建物の構造_ＳＲＣ', '建物の構造_ブロック造', '建物の構造_軽量鉄骨造', '建物の構造_ＲＣ', '建物の構造_木造',
       '建物の構造_鉄骨造', '建物の構造_missing', '取引の事情等_関係者間取引', '取引の事情等_瑕疵有りの可能性',
       '取引の事情等_その他事情有り', '取引の事情等_他の権利・負担付き', '取引の事情等_私道を含む取引', '取引の事情等_隣地の購入',
       '取引の事情等_missing', '取引の事情等_調停・競売等', '用途_工場', '用途_事務所', '用途_住宅', '用途_駐車場',
       '用途_共同住宅', '用途_店舗', '用途_倉庫', '用途_作業場', '用途_missing', '用途_その他'],
      dtype='object')
Index(['id', 'adversarial'], dtype='object')
Index(['最寄駅：距離（分）', '面積（㎡）', '間口', '延床面積（㎡）', '前面道路：幅員（ｍ）', '建ぺい率（％）',
       '容積率（％）', 'y', '間取り_num', '土地の形状_rank', '建築年_西暦', '取引時点_年', '取引時点_四半期',
       '経過年月', '所在地コード_mean_取引年', '所在地コード_std_取引年', '所在地コード_count_取引年',
       '所在地コード_median_取引年', '駅名_mean_取引年', '駅名_std_取引年', '駅名_count_取引年',
       '駅名_median_取引年'],


In [28]:
#NULL_importanceを消す
drop_col = ['adversarial']
train = train.drop(drop_col,axis=1)
test = test.drop(drop_col,axis=1)

In [29]:
cat_features = list(train.select_dtypes('category').columns)

In [30]:
ctb_modelling_params = {
    'learning_rate':[0.1],
    'loss_function':['RMSE'],
    'random_seed': [2020]}

In [31]:
from sklearn.model_selection import ParameterGrid

def make_dataset(df,ID,y):
    X = df.drop([ID,y],axis=1)
    y = df[y]
    return X,ctb.Pool(X,y,cat_features=cat_features)

def ctb_modelling(train,test,option_params,ID,y):
    
    #対数変換
    train[y] = np.log1p(train[y])
    
    #lightGBM用のデータセットに変換
    X_train,train_ctb = make_dataset(train,ID,y)
    X_test,_ = make_dataset(test,ID,y)
    
    #ハイパラの選択肢を作る
    params = list(ParameterGrid(option_params))
    
    #GridSearchを行う
    min_score = np.inf
    for param in tqdm(params):
        result = ctb.cv(params=param,
                        num_boost_round=5000,
                        dtrain = train_ctb,
                        early_stopping_rounds=10,
                        stratified=False,
                        verbose_eval=1000)

        metric = param['loss_function']
        score = result['test-'+metric+'-mean'].min()
        if min_score > score:
            min_score = score
            min_score_iteration = len(result)
            best_hyper_param = param
        
        print(param,min_score_iteration)
        print(score)
    print(min_score,best_hyper_param)
        
    #このハイパラでもう一度学習予測
    bst = ctb.train(params=best_hyper_param,
                    num_boost_round=min_score_iteration,
                    dtrain=train_ctb,
                    verbose_eval=1000)
    
    #テストデータに対して予測をする
    pred_test = bst.predict(X_test)
    pred_test = np.expm1(pred_test)
    pred_test[pred_test<0] == 0
    
    return pred_test,min_score,bst

In [32]:
predict,score,bst = ctb_modelling(train=train,
                                    test=test,
                                    option_params=ctb_modelling_params,
                                    ID='id',
                                    y='y')

0:	learn: 3.3363342	test: 3.3363655	best: 3.3363655 (0)	total: 2.64s	remaining: 3h 40m 16s
1000:	learn: 0.2683995	test: 0.2799213	best: 0.2799213 (1000)	total: 31m 49s	remaining: 2h 7m 10s
2000:	learn: 0.2562369	test: 0.2757341	best: 0.2757341 (2000)	total: 1h 9m 33s	remaining: 1h 44m 14s
3000:	learn: 0.2482839	test: 0.2739901	best: 0.2739901 (3000)	total: 1h 53m 22s	remaining: 1h 15m 31s
4000:	learn: 0.2419505	test: 0.2729312	best: 0.2729312 (4000)	total: 2h 41m 5s	remaining: 40m 13s
Stopped by overfitting detector  (10 iterations wait)
{'learning_rate': 0.1, 'loss_function': 'RMSE', 'random_seed': 2020} 4186
0.2727683879132713

0.2727683879132713 {'learning_rate': 0.1, 'loss_function': 'RMSE', 'random_seed': 2020}
0:	learn: 0.8283323	total: 746ms	remaining: 52m
1000:	learn: 0.2698159	total: 11m 18s	remaining: 35m 57s
2000:	learn: 0.2594124	total: 23m 30s	remaining: 25m 39s
3000:	learn: 0.2527276	total: 36m	remaining: 14m 13s
4000:	learn: 0.2474638	total: 48m 41s	remaining: 2m 15s
418

In [33]:
submission = pd.DataFrame(test['id'])
submission['y'] = predict
display(submission.loc[submission['y']<0,'y'])
submission.loc[submission['y']<0,'y'] = 0
display(submission)

382443   -0.230826
391081   -0.224281
391135   -0.081632
391141   -0.011808
Name: y, dtype: float64

,id,y
356344,1,27.889385
356345,2,14.807780
356346,3,198.027532
356347,4,22.535312
356348,5,60.948851
...,...,...
391183,34840,4.615409
391184,34841,2.203155
391185,34842,2.769293
391186,34843,9.031784


In [34]:
submission.to_csv('../submissions/sub_cat_' + str(round(score,5)) + '.csv',index=False)